In [1]:
import unittest

import torch as th
import numpy as np

from active_critic.model_src.whole_sequence_model import WholeSequenceModel
from active_critic.policy.active_critic_policy import *
from active_critic.utils.test_utils import (make_obs_act_space,
                                            make_wsm_setup)
from active_critic.utils.gym_utils import (DummyExtractor, make_dummy_vec_env,
                                           new_epoch_pap,
                                           new_epoch_reach)

from active_critic.utils.gym_utils import make_policy_dict, new_epoch_reach, make_dummy_vec_env, sample_expert_transitions, parse_sampled_transitions
from active_critic.model_src.state_model import StateModel, StateModelArgs
from active_critic.utils.pytorch_utils import build_tf_horizon_mask

/home/hendrik/miniconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [2]:
from active_critic.model_src.whole_sequence_model import *

def make_acps(seq_len, extractor, new_epoch, batch_size = 32, device='cpu', horizon = 0):
    acps = ActiveCriticPolicySetup()
    acps.device=device
    acps.epoch_len=seq_len
    acps.extractor=extractor
    acps.new_epoch=new_epoch
    acps.opt_steps=100
    acps.inference_opt_lr = 1e-1
    acps.optimizer_class = th.optim.Adam
    acps.optimize = True
    acps.batch_size = batch_size
    acps.pred_mask = build_tf_horizon_mask(seq_len=seq_len, horizon=horizon, device=device)
    acps.opt_mask = th.zeros([batch_size, seq_len, 1], device=device, dtype=bool)
    acps.opt_mask[:,-1] = 1
    return acps

def setup_opt_state(device='cuda'):
    seq_len = 6
    action_dim = 2
    obs_dim = 3
    batch_size = 2
    embed_dim = 4
    lr = 1e-3

    actor_args = StateModelArgs()
    actor_args.arch = [20, action_dim]
    actor_args.device = device
    actor_args.lr = lr
    actor = StateModel(args=actor_args)

    critic_args = StateModelArgs()
    critic_args.arch = [10, 1]
    critic_args.device = device
    critic_args.lr = lr
    critic = StateModel(args=critic_args)

    emitter_args = StateModelArgs()
    emitter_args.arch = [20, embed_dim]
    emitter_args.device = device
    emitter_args.lr = lr
    emitter = StateModel(args=emitter_args)

    predictor_args = make_wsm_setup(
        seq_len=seq_len, d_output=embed_dim, device=device)
    predictor_args.model_setup.d_hid = 200
    predictor_args.model_setup.d_model = 200
    predictor_args.model_setup.nlayers = 1
    predictor = WholeSequenceModel(args=predictor_args)


    acps = make_acps(
        seq_len=seq_len, extractor=DummyExtractor(), new_epoch=new_epoch_pap, device=device, batch_size=batch_size)
    acps.opt_steps = 2
    obs_space, acts_space = make_obs_act_space(
        obs_dim=obs_dim, action_dim=action_dim)
    ac = ActiveCriticPolicy(observation_space=obs_space, 
                            action_space=acts_space,
                            actor=actor,
                            critic=critic,
                            predictor=predictor,
                            emitter=emitter,
                            acps=acps)
    return ac, acps, action_dim, obs_dim, batch_size, embed_dim, seq_len

In [3]:
th.manual_seed(0)
device = 'cpu'
ac, acps, action_dim, obs_dim, batch_size, embed_dim, seq_len = setup_opt_state(device=device)
acps.opt_steps = 100
acps.inference_opt_lr = 1
observation = th.ones([batch_size, 1, obs_dim])
action = ac.predict(observation=observation)

/home/hendrik/miniconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float64
  logger.warn(


In [12]:
assert th.all((ac.history.scores[0][:,1:,-1] - ac.history.scores[0][:,:-1,-1]) >= 0)

In [4]:
ac.history.scores[0].shape

torch.Size([2, 2, 6, 1])

In [5]:
all_actions = action.clone().unsqueeze(1)
all_embeddings = ac.current_embeddings.clone()
expected_action_shape = [batch_size, 1, action_dim]
expected_embedding_shape = [batch_size, 1, embed_dim]
assert list(all_actions.shape) == expected_action_shape
assert list(all_embeddings.shape) == expected_embedding_shape

In [ ]:
for i in range(seq_len-1):
    observation = th.ones([batch_size, 1, obs_dim])
    action = ac.predict(observation=observation)
    assert th.equal(ac.current_actions[:,:-1], all_actions)
    assert th.equal(ac.current_embeddings[:,:-1], all_embeddings)
    all_actions = th.cat((all_actions, action.clone().unsqueeze(1)), dim=1)
    all_embeddings = ac.current_embeddings.clone()

In [ ]:
observation = 2*th.ones([batch_size, 1, obs_dim])
action = ac.predict(observation=observation)

In [ ]:
all_actions = action.clone().unsqueeze(1)
all_embeddings = ac.current_embeddings.clone()
expected_action_shape = [batch_size, 1, action_dim]
expected_embedding_shape = [batch_size, 1, embed_dim]
assert list(all_actions.shape) == expected_action_shape
assert list(all_embeddings.shape) == expected_embedding_shape

In [ ]:
th.manual_seed(0)
device = 'cpu'
ac, acps, action_dim, obs_dim, batch_size, embed_dim, seq_len = setup_opt_state(device=device)
horizon = 0
embeddings = th.ones([batch_size, 1, embed_dim], device=device, requires_grad=True)
actions = th.ones([batch_size, seq_len, action_dim], device=device, requires_grad=True)

mask = build_tf_horizon_mask(seq_len=seq_len, horizon=horizon, device=device)

In [ ]:
th.manual_seed(0)
device = 'cpu'
ac, acps, action_dim, obs_dim, batch_size, embed_dim, seq_len = setup_opt_state(device=device)
horizon = 0
embeddings = th.ones([batch_size, 1, embed_dim], device=device, requires_grad=True)
mask = build_tf_horizon_mask(seq_len=seq_len, horizon=horizon, device=device)
seq_embeddings, actions = ac.build_sequence(embeddings=embeddings, actions=None, seq_len=seq_len, mask=mask)

In [ ]:
th.manual_seed(0)
device = 'cpu'
ac, acps, action_dim, obs_dim, batch_size, embed_dim, seq_len = setup_opt_state(device=device)
horizon = 0
embeddings = th.ones([batch_size, 1, embed_dim], device=device, requires_grad=True)
mask = build_tf_horizon_mask(seq_len=seq_len, horizon=horizon, device=device)
steps = 100
current_step = 2

mask = build_tf_horizon_mask(seq_len=seq_len, horizon=horizon, device=device)
goal_label = th.ones([batch_size, seq_len], device=device, dtype=th.double)
opt_mask = th.zeros_like(goal_label, dtype=th.bool)
opt_mask[:,-current_step:] = 1
loss_reward, actions, seq_embeddings, scores = ac.optimize_sequence(actions=None, seq_embeddings=embeddings, pred_mask=mask, opt_mask=opt_mask, steps=steps, current_step=current_step, goal_label=goal_label)

In [ ]:
loss_reward

In [ ]:
th.manual_seed(0)
device = 'cpu'
ac, acps, action_dim, obs_dim, batch_size, embed_dim, seq_len = setup_opt_state(device=device)
horizon = 0
embeddings = th.ones([batch_size, 1, embed_dim], device=device, requires_grad=True)
actions = th.ones([batch_size, seq_len, action_dim], device=device, requires_grad=True)

mask = build_tf_horizon_mask(seq_len=seq_len, horizon=horizon, device=device)
seq_embeddings, actions = ac.build_sequence(embeddings=embeddings, actions=actions, seq_len=seq_len, mask=mask)
loss = ((seq_embeddings[:,-1] - th.ones_like(seq_embeddings[:,-1] ))**2).mean()
loss.backward()
assert (actions.grad!=0).sum() == actions.numel() - action_dim*batch_size

In [ ]:
horizon = 0
embeddings = th.ones([batch_size, 1, embed_dim], device=device, requires_grad=True)
actions = th.ones([batch_size, seq_len, action_dim], device=device, requires_grad=True)
action_optim = th.optim.Adam([actions], lr=1e-1)
opt_paras = action_optim.state_dict()

mask = build_tf_horizon_mask(seq_len=seq_len, horizon=horizon, device=device)
seq_embeddings, na = ac.build_sequence(embeddings=embeddings, actions=actions, seq_len=seq_len, mask=mask)
opt_actions = actions.detach()
opt_actions.requires_grad = True
next_embedding = ac.predict_step(embeddings=seq_embeddings.detach(), actions=opt_actions, mask=mask)
seq_embedding = th.cat((embeddings[:,:1], next_embedding[:,:-1]), dim=1)
loss = ((seq_embedding[:,-1] - th.ones_like(seq_embedding[:,-1] ))**2).mean()
loss.backward()
assert (opt_actions.grad != 0).sum() == batch_size*action_dim

In [ ]:
th.manual_seed(0)
device = 'cpu'
ac, acps, action_dim, obs_dim, batch_size, embed_dim, seq_len = setup_opt_state(device=device)
horizon = 0
embeddings = th.ones([batch_size, 1, embed_dim], device=device, requires_grad=True)
mask = build_tf_horizon_mask(seq_len=seq_len, horizon=horizon, device=device)
seq_embeddings, na = ac.build_sequence(embeddings=embeddings, actions=actions, seq_len=seq_len, mask=mask)
actions = th.ones([batch_size, seq_len, action_dim], device=device, requires_grad=True)

steps = 1
current_step = 4

mask = build_tf_horizon_mask(seq_len=seq_len, horizon=horizon, device=device)
goal_label = th.ones([batch_size, seq_len], device=device, dtype=th.double)
opt_mask = th.zeros_like(goal_label, dtype=th.bool)
opt_mask[:,-current_step:] = 1
loss_reward, actions, seq_embeddings, scores = ac.optimize_sequence(actions, seq_embeddings, pred_mask=mask, opt_mask=opt_mask, steps=steps, current_step=current_step, goal_label=goal_label)

assert loss_reward > 1e-1

In [ ]:
loss_reward

In [ ]:
th.manual_seed(0)
device = 'cpu'
ac, acps, action_dim, obs_dim, batch_size, embed_dim, seq_len = setup_opt_state(device=device)
horizon = 0
embeddings = th.ones([batch_size, 1, embed_dim], device=device, requires_grad=True)
mask = build_tf_horizon_mask(seq_len=seq_len, horizon=horizon, device=device)
seq_embeddings, na = ac.build_sequence(embeddings=embeddings, actions=actions, seq_len=seq_len, mask=mask)
actions = th.ones([batch_size, seq_len, action_dim], device=device, requires_grad=True)
org_actions = actions.detach().clone()
steps = 100
current_step = 2

mask = build_tf_horizon_mask(seq_len=seq_len, horizon=horizon, device=device)
goal_label = th.ones([batch_size, seq_len], device=device, dtype=th.double)
opt_mask = th.zeros_like(goal_label, dtype=th.bool)
opt_mask[:,-current_step:] = 1
loss_reward, actions, seq_embeddings, scores = ac.optimize_sequence(actions, seq_embeddings, pred_mask=mask, opt_mask=opt_mask, steps=steps, current_step=current_step, goal_label=goal_label)

assert loss_reward < 1e-4

In [ ]:
loss_reward

In [ ]:
assert th.equal(actions[:, :current_step], org_actions[:,:current_step])
assert not th.equal(actions[:,current_step:], org_actions[:,current_step:])

In [ ]:
th.manual_seed(0)
device = 'cpu'
ac, acps, action_dim, obs_dim, batch_size, embed_dim, seq_len = setup_opt_state(device=device)
seq_len = 100
horizon = 0
steps = 100
current_step = 0
embeddings = th.ones([batch_size, 1, embed_dim], device=device, requires_grad=True)
mask = build_tf_horizon_mask(seq_len=seq_len, horizon=horizon, device=device)
seq_embeddings, na = ac.build_sequence(embeddings=embeddings, actions=actions, seq_len=seq_len, mask=mask)
actions = th.ones([batch_size, seq_len, action_dim], device=device, requires_grad=True)
org_actions = actions.detach().clone()

goal_label = th.ones([batch_size, seq_len], device=device, dtype=th.double)
opt_mask = th.zeros_like(goal_label, dtype=th.bool)
opt_mask[:,-1] = 1
acps.optimizer_class = th.optim.Adam
acps.inference_opt_lr = 1e-1
loss_reward_full, actions, seq_embeddings, scores = ac.optimize_sequence(actions, seq_embeddings, pred_mask=mask, opt_mask=opt_mask, steps=steps, current_step=current_step, goal_label=goal_label)

In [ ]:
loss_reward_full

In [ ]:
th.manual_seed(0)
device = 'cpu'
ac, acps, action_dim, obs_dim, batch_size, embed_dim, seq_len = setup_opt_state(device=device)
seq_len = 100
horizon = 0
steps = 100
current_step = 98
embeddings = th.ones([batch_size, 1, embed_dim], device=device, requires_grad=True)
mask = build_tf_horizon_mask(seq_len=seq_len, horizon=horizon, device=device)
seq_embeddings, na = ac.build_sequence(embeddings=embeddings, actions=actions, seq_len=seq_len, mask=mask)
actions = th.ones([batch_size, seq_len, action_dim], device=device, requires_grad=True)
org_actions = actions.detach().clone()

goal_label = th.ones([batch_size, seq_len], device=device, dtype=th.double)
opt_mask = th.zeros_like(goal_label, dtype=th.bool)
opt_mask[:,-1] = 1
acps.optimizer_class = th.optim.Adam
acps.inference_opt_lr = 1e-1
loss_reward_part, actions, seq_embeddings, scores = ac.optimize_sequence(actions, seq_embeddings, pred_mask=mask, opt_mask=opt_mask, steps=steps, current_step=current_step, goal_label=goal_label)

In [ ]:
loss_reward_part

In [ ]:
assert loss_reward_part > loss_reward_full